In [ ]:
import os
import findspark

os.environ["SPARK_HOME"] = "D:/spark-3.5.5-bin-hadoop3"
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()


import pandas as pd

In [ ]:
PANDAS = pd.read_csv('TESTE.csv', sep=';')
SPARK = spark.read.csv('TESTE.csv', sep=';', inferSchema=True)

OBS: os Displays abaixo são apenas para mostrar todos os prints no mesmo output

In [ ]:
# O PRINT DO DATAFRAME DO SPARK É TOTALMENTE DIFERENTE, PORÉM ELE POSSUI UMA FUNÇÃO ESPECIFICA PARA FICAR COMO O DF DO PANDAS:
display(PANDAS)
SPARK.toPandas()

In [ ]:
display(PANDAS.head(5))
SPARK.limit(5).toPandas()

In [ ]:
PANDAS.rename(columns={'TESTE': 'COLUNA_1'}, inplace=True)
SPARK.withColumRenamed('TESTE', 'COLUNA_1')

In [ ]:
print(PANDAS.info)
SPARK.printSchema()

In [ ]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

PANDAS['TESTE2'] = PANDAS['TESTE2'].str.replace(',', '.')
SPARK = SPARK.withColumn('TESTE2', f.regexp_replace('TESTE2', ',', '.'))

In [ ]:
PANDAS['TESTE2'] = PANDAS['TESTE2'].astype(float)
SPARK = SPARK.withColumn('TESTE2', SPARK['TESTE2'].cast(DoubleType))